In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains, ScrollOrigin
import pandas as pd
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
url = 'https://www.thegioididong.com/tai-nghe-co-day#c=54&p=56824&o=7&pi=2'

driver = webdriver.Chrome()
driver.get(url)

### Lấy danh sách sản phẩm

In [3]:
listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')
len(product_list)

45

#### Tạo Func lấy thông 1 tin sản phẩm

In [4]:
def get_product_info(product_elm):
    phone_name = product_elm.find_element(By.XPATH, './/h3').text

    try:
        phone_price = product_elm.find_element(By.XPATH, './/strong[@class="price"]').text
    except:
        phone_price = "N/A"

    
    if phone_price == "N/A":
        try:
            phone_price = product_elm.find_element(By.CLASS_NAME, 'box-price-present').text
        except:
            phone_price = "N/A"
        
    result = {
        'name': phone_name,
        'price': phone_price
    }
    return result

In [6]:
all_reviews = []
tmp = 0

def getReviewAllPhone_v02(product_list):
    global tmp
    i = 0  # Bắt đầu từ 64
    while i < len(product_list):
        # Lấy thông tin sản phẩm và click vào sản phẩm
        product_info = get_product_info(product_list[i])
        item = product_list[i]
        item.click()
        sleep(2)

        try:
            # Nếu có nút "Xem thêm" thì bấm
            read_more_elm = driver.find_element(By.CLASS_NAME, 'btn-view-all')
            if read_more_elm:
                read_more_elm.click()
                sleep(2)

                while True:
                    reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                    for review in reviews:
                        try:
                            all_reviews.append({
                                'product_name': product_info['name'],
                                'product_price': product_info['price'],
                                'review': review.text
                            })
                        except:
                            continue

                    try:
                        page_elm = driver.find_element(By.CLASS_NAME, 'pagcomment')
                        list_page = page_elm.find_elements(By.XPATH, './/a')

                        next_page = None
                        for page in list_page:
                            if page.text.strip() == "»" or page.text.strip() == "›":
                                next_page = page
                                break

                        if next_page:
                            next_page.click()
                            sleep(2)
                        else:
                            last_page = list_page[-1]
                            last_page.click()
                            sleep(2)
                            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                            for review in reviews:
                                try:
                                    all_reviews.append({
                                        'product_name': product_info['name'],
                                        'product_price': product_info['price'],
                                        'review': review.text
                                    })
                                except:
                                    continue
                            driver.back()
                            driver.back()
                            sleep(2)
                            break
                    except:
                        print("Không còn trang tiếp theo!")
                        driver.back()
                        driver.back()
                        sleep(2)
                        break
        except:
            # Nếu chỉ có 1 trang review
            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
            for review in reviews:
                all_reviews.append({
                    'product_name': product_info['name'],
                    'product_price': product_info['price'],
                    'review': review.text
                })
            
            driver.back()
            sleep(2)

        # Sau khi quay lại, làm mới product_list
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'listproduct'))
        )
        listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
        product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')
        print(f"Độ dài product_list sau khi làm mới: {len(product_list)}")
        
        i += 1  # Tăng i để xử lý sản phẩm tiếp theo

In [7]:
getReviewAllPhone_v02(product_list)

Không còn trang tiếp theo!
Độ dài product_list sau khi làm mới: 45
Độ dài product_list sau khi làm mới: 45
Độ dài product_list sau khi làm mới: 45


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6E7C6C085+28581]
	(No symbol) [0x00007FF6E7BD43C0]
	(No symbol) [0x00007FF6E7A691DA]
	(No symbol) [0x00007FF6E7ABF797]
	(No symbol) [0x00007FF6E7ABF9CC]
	(No symbol) [0x00007FF6E7B134C7]
	(No symbol) [0x00007FF6E7AE797F]
	(No symbol) [0x00007FF6E7B10193]
	(No symbol) [0x00007FF6E7AE7713]
	(No symbol) [0x00007FF6E7AB0400]
	(No symbol) [0x00007FF6E7AB16B3]
	GetHandleVerifier [0x00007FF6E7FC2D2D+3529805]
	GetHandleVerifier [0x00007FF6E7FDB9E2+3631362]
	GetHandleVerifier [0x00007FF6E7FD0253+3584371]
	GetHandleVerifier [0x00007FF6E7D36D5A+859258]
	(No symbol) [0x00007FF6E7BDF12F]
	(No symbol) [0x00007FF6E7BDAF94]
	(No symbol) [0x00007FF6E7BDB136]
	(No symbol) [0x00007FF6E7BCAB09]
	BaseThreadInitThunk [0x00007FFEBF53E8D7+23]
	RtlUserThreadStart [0x00007FFEBFE5BF6C+44]


In [25]:
all_reviews

[{'product_name': 'Baseus Lipow 22.5W PPJP312 kèm Cáp Lightning và Type C',
  'product_price': '590.000₫',
  'review': 'Mình mua mẫu này 2 ngày trước. Giờ mình muốn đổi sang mẫu beseus Qpow 2 PPQD420C thì bên mình có hỗ trợ không ạ do lúc mua em đi đường nên chỉ lấy mỗi sạc, phụ kiện đi kèm như hộp em không lấy ạ'},
 {'product_name': 'Xiaomi 22.5W Lite P16ZM',
  'product_price': '390.000₫',
  'review': 'THEGIOIDIDONG bán sản phẩm có nguồn gốc xuất xứ không phải Hàng Face tràn lang trên mạng. Về Sạc dự phòng Thì Đến Từ Hãng XIAOMI sạc rất tốt. Pin của Sạc dự phòng Rất Lâu nếu so sánh sạc dự phòng XIAOMI với Samsung Thì Xiaomi xứng đáng về độ bền và chất lượng. Sản phẩm Đúng như bài đăng'},
 {'product_name': 'Ugreen 20W PB206',
  'product_price': '750.000₫',
  'review': 'Sạc bằng dây cho iphone thì nhanh . Sạc không dây cho iphone thì chậm do không phải magsafe .'},
 {'product_name': 'Ugreen 20W PB206',
  'product_price': '750.000₫',
  'review': 'Đưa hàng cũ'},
 {'product_name': 'Ugreen 

In [ ]:
df = pd.DataFrame(all_reviews)
# Lưu file
df.to_csv("E:\\study\\tu_hoc\\DE\\CRAWL_DATA\\review_data\\data\\head_phone_data.xlsx", index=False)